In [1]:
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import Counter, defaultdict
import pandas as pd
import pickle
import numpy as np
from shapely.geometry import Polygon, Point, LinearRing, box
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn import linear_model, decomposition
from sklearn.feature_extraction import DictVectorizer
%matplotlib notebook

In [88]:
df_train = pd.read_pickle('df_train')
df_test = pd.read_pickle('df_test')
x_train_df = pd.read_pickle('x_train_df_2')
x_test_df = pd.read_pickle('x_test_df_2')

# pick = ['hash','avg_azimuth_12', 'avg_azimuth_13', 'avg_azimuth_14', 'avg_azimuth_15','box_area',
#        'avg_trip_duration_12', 'avg_trip_duration_13', 'avg_trip_duration_14', 'avg_trip_duration_15',
#        'avg_trip_length_12',  'avg_trip_length_13', 'avg_trip_length_14', 'avg_trip_length_15']
# x_train_df = pd.merge(x_train_df,  pd.read_pickle('new_x_train_df')[pick], on='hash')
# x_test_df = pd.merge(x_test_df, pd.read_pickle('new_x_test_df')[pick], on='hash')

# x_train_df = pd.merge(x_train_df, pd.read_csv("data_trip_city_buffer_train.csv"), on='hash',how='left').fillna(0)


print(x_train_df.shape, x_test_df.shape)

(134063, 119) (33515, 119)


In [89]:
training_dataset = x_train_df.drop(['hash','trajectory_id'], axis=1)
testing_dataset = x_test_df.drop(['hash','trajectory_id'], axis=1)
x_1 = training_dataset.drop(['label'], axis=1).values
y_1 = training_dataset['label'].values
sc_1 = StandardScaler()
x_1 = sc_1.fit_transform(x_1)

In [104]:
xTrain, xTest, yTrain, yTest = train_test_split(x_1, y_1, test_size = 0.0, random_state = 0)
clf_1 = RandomForestClassifier(n_jobs = -1, criterion = 'entropy',n_estimators = 500, max_depth = 60,
                               max_features = 'sqrt', random_state=42)
# clf_1 = AdaBoostClassifier(n_estimators=100)
# clf_1 = MLPClassifier(activation= 'relu',hidden_layer_sizes=(30,30,30))
# clf_1 = DecisionTreeClassifier(max_depth=500)
clf_1.fit(xTrain, yTrain)
# yPred = clf_1.predict(xTest)
# print(classification_report(yTest, yPred))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=60, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [ ]:
features = training_dataset.drop(['label'], axis=1).columns.values
importances = clf_1.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(xTrain.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]), features[indices[f]])

In [105]:
x_test_df['label'] =  clf_1.predict(sc_1.fit_transform(testing_dataset.drop(['label'], axis=1).values))

In [108]:
x_test_df.label.value_counts()

0    25258
1     8257
Name: label, dtype: int64

In [107]:
x_test_df['label'] = x_test_df.apply(lambda row: 0 
                                 if ((row['duration'] == 0.0) &
                                     ~(((row['x_entry'] >= 3750901.5068) &
                                                               (row['x_entry'] <= 3770901.5068)) &
                                       ((row['y_entry'] >= -19268905.6133) &
                                        (row['y_entry'] <= -19208905.6133))))
                                 else row['label'], axis=1)
x_test_df['label'] = x_test_df.apply(lambda row: 1
                                 if ((row['duration'] == 0.0) &
                                     (((row['x_entry'] >= 3750901.5068) &
                                                               (row['x_entry'] <= 3770901.5068)) &
                                       ((row['y_entry'] >= -19268905.6133) &
                                        (row['y_entry'] <= -19208905.6133))))
                                 else row['label'], axis=1)

In [109]:
x_test_df.sort_values('trajectory_id',ascending=True)[['trajectory_id','label']].to_csv('output40.csv', index=False)